In [ ]:
from __future__ import print_function
import os
import time
import logging
import argparse
from visdom import Visdom
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *
import dataset

# Teacher models
import models

# Student models


start_time = time.time()
# os.makedirs('./checkpoint', exist_ok=True)

# Training settings
parser = argparse.ArgumentParser(description='PyTorch original KD')
parser.add_argument('--dataset',
                    choices=['CIFAR10',
                             'CIFAR100',
                             'tinyimagenet'
                            ],
                    default='CIFAR10')
parser.add_argument('--teacher',
                    choices=['ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default='ResNet110')
parser.add_argument('--student',
                    choices=[
                             'ResNet8',
                             'ResNet15',
                             'ResNet16',
                             'ResNet20',
                             'myNet'
                            ],
                    default='ResNet20')
parser.add_argument('--n_class', type=int, default=10, metavar='N', help='num of classes')
parser.add_argument('--T', type=float, default=20.0, metavar='Temputure', help='Temputure for distillation')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--test_batch_size', type=int, default=128, metavar='N', help='input test batch size for training')
parser.add_argument('--epochs', type=int, default=20, metavar='N', help='number of epochs to train (default: 20)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum (default: 0.5)')
parser.add_argument('--device', default='cuda:1', type=str, help='device: cuda or cpu')
parser.add_argument('--print_freq', type=int, default=10, metavar='N', help='how many batches to wait before logging training status')

config = ['--dataset', 'CIFAR100', '--epochs', '200', '--n_class', '100', '--teacher', 'ResNet110', '--student', 'ResNet8', '--T', '5.0', '--device', 'cuda:0']
args = parser.parse_args(config)

device = args.device if torch.cuda.is_available() else 'cpu'
load_dir = './checkpoint/' + args.dataset + '/'

teacher_model = getattr(models, args.teacher)(args.n_class)
teacher_model.load_state_dict(torch.load(load_dir + args.teacher + '.pth'))
teacher_model.to(device)

st_model = getattr(models, args.student)(args.n_class)  # args.student()
st_model.to(device)

# logging
logfile = load_dir + 'KD_' + st_model.model_name + '.log'
if os.path.exists(logfile):
    os.remove(logfile)
def log_out(info):
    f = open(logfile, mode='a')
    f.write(info)
    f.write('\n')
    f.close()
    print(info)
    
# visualizer
vis = Visdom(env='distill')
loss_win = vis.line(
    X=np.array([0]),
    Y=np.array([0]),
    opts=dict(
        title=args.student + ' KD Loss',
        xtickmin=0,
#         xtickmax=1,
#         xtickstep=5,
        ytickmin=0,
#         ytickmax=10,
#         ytickstep=5,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
    ),
    name="loss"
)
    
acc_win = vis.line(
    X=np.column_stack((0, 0)),
    Y=np.column_stack((0, 0)),
    opts=dict(
        title=args.student + ' KD ACC',
        xtickmin=0,
#         xtickstep=5,
        ytickmin=0,
        ytickmax=100,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
        legend=['train_acc', 'test_acc']
    ),
    name="acc"
)


# data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([transforms.ToTensor(), normalize])
if args.dataset == 'tinyimagenet':
    train_set = dataset.TinyImageNet(root='../data/tiny-imagenet-200', transform=train_transform)
    test_set = dataset.TinyImageNet(root='../data/tiny-imagenet-200', transform=test_transform)
else:
    train_set = getattr(datasets, args.dataset)(root='../data', train=True, download=True, transform=train_transform)
    test_set = getattr(datasets, args.dataset)(root='../data', train=False, download=False, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.test_batch_size, shuffle=False)

# optimizer = optim.SGD(st_model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer_sgd = optim.SGD(st_model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_sgd, milestones=[100, 150])

def distillation(y, labels, teacher_scores, T, alpha):
    return nn.KLDivLoss()(F.log_softmax(y/T), F.softmax(teacher_scores/T)) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)


def train(epoch, model, loss_fn):
    print('Training:')
    # switch to train mode
    model.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.to(device), target.to(device)
        optimizer_sgd.zero_grad()
        # compute outputs
        _,_,_,_, output = model(input)
        with torch.no_grad():
            _,_,_,_, t_output = teacher_model(input)

#         print(output.size(), target.size(), teacher_output.size())

        # compute gradient and do SGD step
        loss = loss_fn(output, target, t_output, T=args.T, alpha=0.7)

        loss.backward()
        optimizer_sgd.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        train_acc = accuracy(output.data, target.data)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(train_acc, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_out('[{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
    return losses.avg, train_acc.cpu().numpy()


def test(model):
    print('Testing:')
    # switch to evaluate mode
    model.eval()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)

            # compute output
            _,_,_,_, output = model(input)
            loss = F.cross_entropy(output, target)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            test_acc = accuracy(output.data, target.data)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(test_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                log_out('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    log_out(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()


print('StudentNet:\n')
print(st_model)
best_acc = 0
for epoch in range(1, args.epochs + 1):
    log_out("\n===> epoch: {}/{}".format(epoch, args.epochs))
    log_out('current lr {:.5e}'.format(optimizer_sgd.param_groups[0]['lr']))
    lr_scheduler.step()
    train_loss, train_acc = train(epoch, st_model, loss_fn=distillation)
    # visaulize loss
    vis.line(np.array([train_loss]), np.array([epoch]), loss_win, update="append")
    _, test_acc, top1 = test(st_model)
    best_acc = max(top1, best_acc)
    vis.line(np.column_stack((train_acc, top1)), np.column_stack((epoch, epoch)), acc_win, update="append")

# torch.save(st_model.state_dict(), load_dir + args.teacher + '_distill_' + args.student + '.pth')
# release GPU memory
torch.cuda.empty_cache()

log_out("@ BEST ACC = {:.4f}%".format(best_acc))
log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))


/home/data/yaliu/jupyterbooks/multi-KD/models/teacher/resnet20.py:36: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)
/home/data/yaliu/jupyterbooks/multi-KD/models/student/resnet_s.py:13: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


Files already downloaded and verified
StudentNet:

ResNet(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:146: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:146: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:1992: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


[0/391]	Time 1.085 (1.085)	Data 0.071 (0.071)	Loss 1.7962 (1.7962)	Prec@1 0.781 (0.781)
[10/391]	Time 0.086 (0.186)	Data 0.043 (0.047)	Loss 1.6153 (1.7022)	Prec@1 0.781 (0.781)
[20/391]	Time 0.066 (0.135)	Data 0.020 (0.040)	Loss 1.6127 (1.6640)	Prec@1 0.781 (1.042)
[30/391]	Time 0.076 (0.121)	Data 0.034 (0.039)	Loss 1.5903 (1.6418)	Prec@1 1.562 (1.134)
[40/391]	Time 0.079 (0.112)	Data 0.034 (0.038)	Loss 1.5770 (1.6285)	Prec@1 0.781 (1.086)
[50/391]	Time 0.101 (0.107)	Data 0.053 (0.039)	Loss 1.5876 (1.6184)	Prec@1 0.781 (1.195)
[60/391]	Time 0.073 (0.103)	Data 0.029 (0.038)	Loss 1.5613 (1.6117)	Prec@1 0.781 (1.242)
[70/391]	Time 0.070 (0.100)	Data 0.024 (0.037)	Loss 1.5909 (1.6068)	Prec@1 1.562 (1.309)
[80/391]	Time 0.094 (0.099)	Data 0.051 (0.038)	Loss 1.5648 (1.6026)	Prec@1 1.562 (1.360)
[90/391]	Time 0.101 (0.098)	Data 0.037 (0.038)	Loss 1.5920 (1.5998)	Prec@1 0.781 (1.339)
[100/391]	Time 0.082 (0.096)	Data 0.035 (0.038)	Loss 1.5798 (1.5967)	Prec@1 1.562 (1.377)
[110/391]	Time 0.069 

Test: [50/79]	Time 0.037 (0.036)	Loss 3.7433 (3.6788)	Prec@1 10.156 (11.535)
Test: [60/79]	Time 0.032 (0.036)	Loss 3.9528 (3.6883)	Prec@1 8.594 (11.399)
Test: [70/79]	Time 0.036 (0.036)	Loss 3.5814 (3.6869)	Prec@1 12.500 (11.202)
 * Prec@1 11.230

===> epoch: 3/200
current lr 1.00000e-01
Training:
[0/391]	Time 0.092 (0.092)	Data 0.046 (0.046)	Loss 1.2542 (1.2542)	Prec@1 14.062 (14.062)
[10/391]	Time 0.084 (0.083)	Data 0.043 (0.037)	Loss 1.2873 (1.2636)	Prec@1 11.719 (12.642)
[20/391]	Time 0.075 (0.082)	Data 0.032 (0.035)	Loss 1.2615 (1.2599)	Prec@1 12.500 (12.760)
[30/391]	Time 0.086 (0.083)	Data 0.035 (0.035)	Loss 1.2347 (1.2566)	Prec@1 15.625 (12.853)
[40/391]	Time 0.119 (0.085)	Data 0.038 (0.036)	Loss 1.2820 (1.2545)	Prec@1 11.719 (12.919)
[50/391]	Time 0.073 (0.085)	Data 0.027 (0.036)	Loss 1.2528 (1.2542)	Prec@1 14.062 (12.791)
[60/391]	Time 0.080 (0.086)	Data 0.032 (0.036)	Loss 1.2418 (1.2517)	Prec@1 10.156 (12.666)
[70/391]	Time 0.102 (0.087)	Data 0.057 (0.037)	Loss 1.3338 (1.249

...

[160/391]	Time 0.101 (0.113)	Data 0.037 (0.037)	Loss 0.5836 (0.5337)	Prec@1 57.812 (57.356)
[170/391]	Time 0.123 (0.113)	Data 0.034 (0.037)	Loss 0.5507 (0.5339)	Prec@1 57.031 (57.401)
[180/391]	Time 0.130 (0.113)	Data 0.043 (0.037)	Loss 0.6125 (0.5342)	Prec@1 48.438 (57.247)
[190/391]	Time 0.089 (0.113)	Data 0.036 (0.037)	Loss 0.5835 (0.5358)	Prec@1 56.250 (57.113)
[200/391]	Time 0.082 (0.112)	Data 0.036 (0.037)	Loss 0.5458 (0.5360)	Prec@1 60.156 (57.121)
[210/391]	Time 0.105 (0.113)	Data 0.034 (0.037)	Loss 0.5293 (0.5359)	Prec@1 53.125 (57.105)
[220/391]	Time 0.121 (0.113)	Data 0.037 (0.037)	Loss 0.5528 (0.5358)	Prec@1 55.469 (57.134)
[230/391]	Time 0.112 (0.113)	Data 0.059 (0.038)	Loss 0.5232 (0.5351)	Prec@1 54.688 (57.204)
[240/391]	Time 0.127 (0.113)	Data 0.072 (0.038)	Loss 0.5383 (0.5353)	Prec@1 58.594 (57.200)
[250/391]	Time 0.099 (0.112)	Data 0.039 (0.038)	Loss 0.5317 (0.5359)	Prec@1 59.375 (57.115)
[260/391]	Time 0.110 (0.112)	Data 0.034 (0.038)	Loss 0.5382 (0.5359)	Prec@1 57.0

[110/391]	Time 0.078 (0.116)	Data 0.027 (0.039)	Loss 0.5492 (0.5340)	Prec@1 56.250 (57.278)
[120/391]	Time 0.107 (0.116)	Data 0.039 (0.038)	Loss 0.4858 (0.5337)	Prec@1 57.031 (57.309)
[130/391]	Time 0.134 (0.116)	Data 0.035 (0.039)	Loss 0.5510 (0.5352)	Prec@1 60.156 (57.240)
[140/391]	Time 0.152 (0.117)	Data 0.059 (0.039)	Loss 0.5347 (0.5340)	Prec@1 59.375 (57.247)
[150/391]	Time 0.101 (0.116)	Data 0.054 (0.039)	Loss 0.5264 (0.5336)	Prec@1 58.594 (57.212)
[160/391]	Time 0.109 (0.115)	Data 0.046 (0.039)	Loss 0.5748 (0.5339)	Prec@1 57.812 (57.167)
[170/391]	Time 0.113 (0.115)	Data 0.035 (0.039)	Loss 0.5627 (0.5351)	Prec@1 54.688 (57.050)
[180/391]	Time 0.076 (0.115)	Data 0.030 (0.039)	Loss 0.5126 (0.5352)	Prec@1 57.812 (56.979)
[190/391]	Time 0.128 (0.115)	Data 0.038 (0.039)	Loss 0.5604 (0.5359)	Prec@1 53.906 (56.876)
[200/391]	Time 0.107 (0.115)	Data 0.043 (0.040)	Loss 0.5002 (0.5361)	Prec@1 62.500 (56.880)
[210/391]	Time 0.133 (0.115)	Data 0.040 (0.040)	Loss 0.5667 (0.5363)	Prec@1 57.0

[60/391]	Time 0.126 (0.114)	Data 0.032 (0.037)	Loss 0.5760 (0.5446)	Prec@1 57.031 (56.032)
[70/391]	Time 0.171 (0.116)	Data 0.076 (0.038)	Loss 0.5362 (0.5416)	Prec@1 53.906 (56.349)
[80/391]	Time 0.137 (0.115)	Data 0.047 (0.038)	Loss 0.6060 (0.5441)	Prec@1 61.719 (56.404)
[90/391]	Time 0.084 (0.113)	Data 0.028 (0.038)	Loss 0.6529 (0.5458)	Prec@1 43.750 (56.198)
[100/391]	Time 0.079 (0.114)	Data 0.034 (0.038)	Loss 0.5340 (0.5428)	Prec@1 57.812 (56.559)
[110/391]	Time 0.101 (0.113)	Data 0.039 (0.038)	Loss 0.5637 (0.5420)	Prec@1 54.688 (56.560)
[120/391]	Time 0.115 (0.114)	Data 0.030 (0.038)	Loss 0.4857 (0.5408)	Prec@1 58.594 (56.657)
[130/391]	Time 0.094 (0.113)	Data 0.027 (0.038)	Loss 0.4792 (0.5386)	Prec@1 60.156 (56.787)
[140/391]	Time 0.112 (0.114)	Data 0.034 (0.039)	Loss 0.5525 (0.5394)	Prec@1 57.031 (56.671)
[150/391]	Time 0.135 (0.114)	Data 0.034 (0.039)	Loss 0.4894 (0.5391)	Prec@1 64.062 (56.716)
[160/391]	Time 0.095 (0.115)	Data 0.032 (0.039)	Loss 0.4670 (0.5396)	Prec@1 65.625 (

[10/391]	Time 0.105 (0.106)	Data 0.037 (0.035)	Loss 0.5054 (0.5383)	Prec@1 57.812 (56.818)
[20/391]	Time 0.129 (0.110)	Data 0.034 (0.035)	Loss 0.5125 (0.5384)	Prec@1 62.500 (57.292)
[30/391]	Time 0.118 (0.112)	Data 0.034 (0.035)	Loss 0.5162 (0.5274)	Prec@1 60.156 (58.695)
[40/391]	Time 0.096 (0.111)	Data 0.031 (0.035)	Loss 0.5695 (0.5342)	Prec@1 53.125 (57.851)
[50/391]	Time 0.105 (0.111)	Data 0.030 (0.034)	Loss 0.5337 (0.5332)	Prec@1 60.156 (58.134)
[60/391]	Time 0.115 (0.110)	Data 0.038 (0.034)	Loss 0.4523 (0.5356)	Prec@1 64.062 (57.812)
[70/391]	Time 0.134 (0.111)	Data 0.038 (0.036)	Loss 0.4826 (0.5353)	Prec@1 62.500 (57.570)
[80/391]	Time 0.107 (0.110)	Data 0.027 (0.035)	Loss 0.4807 (0.5349)	Prec@1 57.031 (57.436)
[90/391]	Time 0.128 (0.111)	Data 0.031 (0.035)	Loss 0.5247 (0.5362)	Prec@1 60.156 (57.315)
[100/391]	Time 0.093 (0.111)	Data 0.031 (0.035)	Loss 0.5008 (0.5346)	Prec@1 58.594 (57.433)
[110/391]	Time 0.104 (0.111)	Data 0.030 (0.035)	Loss 0.5178 (0.5350)	Prec@1 59.375 (57.34

Test: [40/79]	Time 0.028 (0.038)	Loss 1.4470 (1.6781)	Prec@1 61.719 (53.563)
Test: [50/79]	Time 0.035 (0.037)	Loss 1.8787 (1.6798)	Prec@1 49.219 (53.278)
Test: [60/79]	Time 0.034 (0.036)	Loss 2.1127 (1.6861)	Prec@1 46.875 (53.112)
Test: [70/79]	Time 0.035 (0.036)	Loss 1.7445 (1.6857)	Prec@1 53.906 (53.202)
 * Prec@1 53.390

===> epoch: 147/200
current lr 1.00000e-02
Training:
[0/391]	Time 0.119 (0.119)	Data 0.056 (0.056)	Loss 0.5099 (0.5099)	Prec@1 56.250 (56.250)
[10/391]	Time 0.139 (0.129)	Data 0.077 (0.053)	Loss 0.4991 (0.5284)	Prec@1 61.719 (57.528)
[20/391]	Time 0.076 (0.125)	Data 0.026 (0.046)	Loss 0.5393 (0.5202)	Prec@1 50.781 (58.222)
[30/391]	Time 0.125 (0.118)	Data 0.032 (0.042)	Loss 0.5583 (0.5235)	Prec@1 54.688 (58.115)
[40/391]	Time 0.120 (0.117)	Data 0.033 (0.041)	Loss 0.5368 (0.5275)	Prec@1 53.906 (57.679)
[50/391]	Time 0.136 (0.117)	Data 0.049 (0.040)	Loss 0.5488 (0.5285)	Prec@1 54.688 (57.506)
[60/391]	Time 0.111 (0.115)	Data 0.026 (0.039)	Loss 0.5197 (0.5322)	Prec@1 5